In [1]:
import pandas as pd
import numpy as np
import janitor

### Unseen Intakes

In [2]:
unseen_intakes = pd.read_csv('https://data.austintexas.gov/api/views/wter-evkm/rows.csv?accessType=DOWNLOAD')

In [3]:
unseen_intakes = unseen_intakes.clean_names()

#### Subset data to dogs and only get new data

In [4]:
unseen_intakes = unseen_intakes[unseen_intakes['animal_type'] == 'Dog']

In [5]:
unseen_intakes['datetime'] = pd.to_datetime(unseen_intakes['datetime'])

In [6]:
unseen_intakes = unseen_intakes[unseen_intakes['datetime'].dt.date > pd.to_datetime('2021-05-16')]

#### Add year and month columns

In [7]:
unseen_intakes['intake_year'] = unseen_intakes['datetime'].dt.year
unseen_intakes['intake_month'] = unseen_intakes['datetime'].dt.month

#### Get sex and spay/neuter split

In [8]:
unseen_intakes['sex_upon_intake'] = unseen_intakes['sex_upon_intake'].str.replace('Unknown', 'Unknown Unknown')
unseen_intakes['sex_split'] = unseen_intakes['sex_upon_intake'].str.split(' ')
unseen_intakes['spay_neuter'] = unseen_intakes['sex_upon_intake'].str.split(' ').str[0]
unseen_intakes['sex'] = unseen_intakes['sex_upon_intake'].str.split(' ').str[1]

#### Age in Years

In [9]:
#age digit
unseen_intakes['age_digit'] = [str.split(x, ' ')[0] for x in unseen_intakes['age_upon_intake']]
unseen_intakes['age_digit'] = unseen_intakes['age_digit'].str.replace('-', '')
unseen_intakes['age_digit'] = unseen_intakes['age_digit'].astype('int')

#age unit
unseen_intakes['age_unit'] = [str.split(x, ' ')[1] for x in unseen_intakes['age_upon_intake']]
age_unit_dict = {
    'year':'years',
    'month':'months',
    'week': 'weeks',
    'day':'days'
}

unseen_intakes['age_unit'] = unseen_intakes['age_unit'].map(age_unit_dict).fillna(unseen_intakes['age_unit'])

In [10]:
def age_in_years(unit):
    if unit == 'years':
        return 1
    elif unit == 'months':
        return 12
    elif unit == 'weeks':
        return 52.143
    elif unit == 'days':
        return 365

In [11]:
unseen_intakes['age_conversion'] = [age_in_years(x) for x in unseen_intakes['age_unit']]
unseen_intakes['age_in_years'] = round(unseen_intakes['age_digit'] / unseen_intakes['age_conversion'], 2)

#### Breed

In [12]:
unseen_intakes['breed'] = unseen_intakes['breed'].str.replace('Black/Tan Hound', 'Black and Tan Hound')

In [13]:
#Mixed breeds either contain "Mix" or "/"
unseen_intakes['mix_flag'] = np.where((unseen_intakes['breed'].str.contains(' Mix')) | (unseen_intakes['breed'].str.contains('\/')), True, False)

#### Category

In [14]:
unseen_intakes['category'] = 'intake'

In [15]:
columns_to_keep = [
    'animal_id',
    'intake_year',
    'intake_month',
    'datetime',
    'intake_type',
    'intake_condition',
    'spay_neuter',
    'age_in_years',
    'breed',
    'category'
]

unseen_intakes = unseen_intakes[columns_to_keep]

### Unseen Outcomes

In [16]:
unseen_outcomes = pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')

In [17]:
unseen_outcomes = unseen_outcomes.clean_names()

#### Subset data

In [18]:
unseen_outcomes = unseen_outcomes[unseen_outcomes['animal_type'] == 'Dog']

In [19]:
unseen_outcomes['datetime'] = pd.to_datetime(unseen_outcomes['datetime'])

In [20]:
unseen_outcomes = unseen_outcomes[unseen_outcomes['datetime'].dt.date > pd.to_datetime('2021-05-16')]

#### Category

In [21]:
unseen_outcomes['category'] = 'outcome'

#### Selecting columns

In [22]:
unseen_outcomes = unseen_outcomes[['animal_id', 'datetime', 'outcome_type', 'category']]

### Merge

In [23]:
i_merge = unseen_intakes[['animal_id', 'datetime', 'category']]
o_merge = unseen_outcomes[['animal_id', 'datetime', 'category']]

In [24]:
intermediate = pd.concat([i_merge, o_merge]).sort_values('datetime')

In [25]:
intermediate['time_shift'] = intermediate.groupby('animal_id').datetime.shift(-1)
intermediate['next_event'] = intermediate.groupby('animal_id').category.shift(-1)

In [26]:
intermediate

,animal_id,datetime,category,time_shift,next_event
125926,A834676,2021-05-17 08:55:00,intake,2021-05-18 11:29:00,outcome
125896,A834682,2021-05-17 09:33:00,intake,2021-05-20 11:51:00,outcome
125897,A834683,2021-05-17 09:33:00,intake,2021-05-20 11:51:00,outcome
126860,A834472,2021-05-17 09:39:00,outcome,NaT,NaN
126366,A834609,2021-05-17 11:30:00,outcome,NaT,NaN
...,...,...,...,...,...
127076,A836125,2021-06-08 13:05:00,outcome,NaT,NaN
127078,A835508,2021-06-08 14:10:00,outcome,NaT,NaN
127081,A835502,2021-06-08 14:10:00,outcome,NaT,NaN
126878,A836297,2021-06-08 14:15:00,intake,NaT,NaN


In [27]:
intermediate = intermediate[(intermediate['category'] == 'intake') & (intermediate['next_event'] == 'outcome')]

In [28]:
intermediate

,animal_id,datetime,category,time_shift,next_event
125926,A834676,2021-05-17 08:55:00,intake,2021-05-18 11:29:00,outcome
125896,A834682,2021-05-17 09:33:00,intake,2021-05-20 11:51:00,outcome
125897,A834683,2021-05-17 09:33:00,intake,2021-05-20 11:51:00,outcome
126058,A833321,2021-05-17 12:58:00,intake,2021-05-20 12:57:00,outcome
125906,A834710,2021-05-17 13:10:00,intake,2021-05-20 11:24:00,outcome
...,...,...,...,...,...
126860,A836087,2021-06-05 15:02:00,intake,2021-06-05 16:02:00,outcome
126857,A836090,2021-06-05 15:23:00,intake,2021-06-07 12:00:00,outcome
126942,A836112,2021-06-06 10:05:00,intake,2021-06-07 12:01:00,outcome
127026,A836125,2021-06-06 11:36:00,intake,2021-06-08 13:05:00,outcome


In [29]:
intermediate = intermediate.merge(unseen_intakes, on=['animal_id', 'datetime'], suffixes=['_inter', '_intakes'])

In [30]:
unseen_merged = intermediate.merge(unseen_outcomes, left_on=['animal_id', 'time_shift'], right_on=['animal_id', 'datetime'])

#### Subset to "Normal" intake condition

In [31]:
unseen_merged = unseen_merged[unseen_merged['intake_condition'] == 'Normal']

#### Rename Columns

In [32]:
unseen_merged = unseen_merged.rename(columns={'spay_neuter':'intake_spay_neuter', 
                                              'breed_x':'breed',
                                             'age_in_years':'intake_age_in_years'})

In [33]:
unseen_merged.columns

Index(['animal_id', 'datetime_x', 'category_inter', 'time_shift', 'next_event',
       'intake_year', 'intake_month', 'intake_type', 'intake_condition',
       'intake_spay_neuter', 'intake_age_in_years', 'breed',
       'category_intakes', 'datetime_y', 'outcome_type', 'category'],
      dtype='object')

#### Select Columns

In [35]:
unseen_merged = unseen_merged[['intake_type', 'intake_year', 'intake_month', 'intake_spay_neuter', 'breed', 'intake_age_in_years', 'outcome_type']]

In [36]:
unseen_merged.to_csv('../data/unseen_data', index=False)